headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6','Cache-Control': 'max-age=0','Connection': 'keep-alive','Cookie': 'frombot=1; DM5_MACHINEKEY=667603e3-f455-4ec2-8b45-5d2c1a7ce73f; SERVERID=api03; UM_distinctid=1733820c1f1181-08ea685a98e3d2-4353760-384000-1733820c1f288e; dm5_newsearch=%5b%7b%22Title%22%3a%22%e9%9b%bb%e9%8b%b8%e4%ba%ba%22%2c%22Url%22%3a%22%5c%2fsearch%3ftitle%3d%25E9%259B%25BB%25E9%258B%25B8%25E4%25BA%25BA%26language%3d1%22%7d%2c%7b%22Title%22%3a%22%e7%94%b5%e9%94%af%e4%ba%ba%22%2c%22Url%22%3a%22%5c%2fsearch%3ftitle%3d%25E7%2594%25B5%25E9%2594%25AF%25E4%25BA%25BA%26language%3d1%22%7d%5d; ComicHistoryitem_zh=; CNZZDATA1257110450=222295925-1594371969-https%253A%252F%252Fwww.google.com%252F%7C1594374141','Host': 'www.manhuaren.com','If-Modified-Since': 'Friday, 10 July 2020 11:00:45','If-None-Match': '637300044453577807-0---0-2_20-3','Sec-Fetch-Dest': 'document','Sec-Fetch-Mode': 'navigate','Sec-Fetch-Site': 'none','Sec-Fetch-User': '?1','Upgrade-Insecure-Requests': '1','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Cookie': 'frombot=1; DM5_MACHINEKEY=667603e3-f455-4ec2-8b45-5d2c1a7ce73f; SERVERID=api03; UM_distinctid=1733820c1f1181-08ea685a98e3d2-4353760-384000-1733820c1f288e; dm5_newsearch=%5b%7b%22Title%22%3a%22%e9%9b%bb%e9%8b%b8%e4%ba%ba%22%2c%22Url%22%3a%22%5c%2fsearch%3ftitle%3d%25E9%259B%25BB%25E9%258B%25B8%25E4%25BA%25BA%26language%3d1%22%7d%2c%7b%22Title%22%3a%22%e7%94%b5%e9%94%af%e4%ba%ba%22%2c%22Url%22%3a%22%5c%2fsearch%3ftitle%3d%25E7%2594%25B5%25E9%2594%25AF%25E4%25BA%25BA%26language%3d1%22%7d%5d; ComicHistoryitem_zh=; CNZZDATA1257110450=222295925-1594371969-https%253A%252F%252Fwww.google.com%252F%7C1594374141',
    'Host': 'www.manhuaren.com',
    'If-Modified-Since': 'Friday, 10 July 2020 11:00:45',
    'If-None-Match': '637300044453577807-0---0-2_20-3',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

In [ ]:
import requests

headers = {'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6'}

urlBase = 'https://www.manhuaren.com'
url = 'https://www.manhuaren.com/manhua-haizeiwang-onepiece/?from=/manhua-list/'
res = requests.get(url, headers = headers)
#res.encoding = 'UTF-8'
print(res.text)

In [ ]:
from bs4 import BeautifulSoup
from opencc import OpenCC
import lxml

soup = BeautifulSoup(res.text, 'lxml')
liList = soup.findAll("a",{"class":"chapteritem"})
comic_title = OpenCC('s2twp').convert(soup.findAll("p",{"class":"detail-main-info-title"})[0].text)

In [ ]:
#liList
comic_title

In [ ]:
from opencc import OpenCC
comics = []
links = []
for li in liList:
    #檢體轉繁體
    title_ch = li.text
    title_tw = OpenCC('s2twp').convert(title_ch)
    comics.append([title_tw, title_ch, urlBase+li.get('href')])
    #links.append(urlBase+li.get('href'))
comics.reverse()

In [ ]:
comics

In [ ]:
import pandas
comic_df = pandas.DataFrame(comics, columns = ['series-tw', 'series-cn', 'link'])
#print(comic_df)
#comic_df.columns = ['series', 'link']
comic_df

In [ ]:
import sqlite3 as lite
with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
    comic_df.to_sql('comics', con = db, if_exists='replace')
comic_df

In [ ]:
with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
    df2 = pandas.read_sql_query('select "series-tw","series-cn" from comics', con = db)
df2

In [ ]:
from bs4 import BeautifulSoup
from opencc import OpenCC
import sqlite3 as lite
import numpy as np
import pandas, requests



def check_comics():
    # 檢查資料庫最新的漫畫集數
    with lite.connect('./sqlite/comics.sqlite') as db:
        dbCheck = pandas.read_sql_query('select series from comics', con = db)

    datas = []
    with lite.connect('./sqlite/comics.sqlite') as db:
        datas = pandas.read_sql_query('select series, link from comics', con = db)
  
    print(datas)
    
    
    # 利用網路爬蟲檢查網路上最新的集數
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    new_comics = []
    for rec in soup.findAll('a', {'class', 'chapteritem'}):
        title = rec.text
        #title = OpenCC('s2twp').convert(rec.text)
        if not (title in dbCheck.values):
            new_comics.append([title,urlBase+rec.get('href')])
            print("新增"+title)
    new_comics.reverse()
     
    if not(new_comics == []):
        comics = np.append(datas.values,np.array(new_comics),0)
        comic_df = pandas.DataFrame(comics, columns = ['series', 'link'])
        with lite.connect('./sqlite/comics.sqlite') as db:
            comic_df.to_sql('comics', con = db, if_exists='replace')
    #print(new_comics)
    
    return new_comics

#test
"""
new_data = check_comics()
if new_data == []:
    print('沒有更新')
else:
    print(np.array(new_data))

with lite.connect('./sqlite/comics.sqlite') as db:
    alldata = pandas.read_sql_query('select * from comics', con = db)
print("所有資料\n")
print(alldata)
"""

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests, re, time, os, shutil


def getImgLink(link):
    driver = webdriver.Chrome('C:\\Users\\axuy312\\Anaconda_BigData\\高階驗證碼\\driver\\chromedriver.exe')
    driver.get(link)
    imgLinks = []
    eles = driver.find_elements_by_class_name('lazy')
    for e in eles:
        imgLinks.append(e.get_attribute('src'))
    #print(imgLink)
    driver.close()
    print()
    return imgLinks

def getComic(link, series):
    res = requests.get(link, headers=headers)
    #print(res.text)
    
    print('更新中....')
    soup = BeautifulSoup(res.text, 'lxml')
    scriptList = soup.findAll("script")
    
    cnt = 0
    MaxCnt = 0
    Max = 0
    parseString = ''
    for script in scriptList:
        string = script.find(text=True, recursive=False)
        if not(string == None) and len(string) > Max:
            Max = len(string)
            MaxCnt = cnt
            parseString = string
        cnt = cnt + 1
    initLinks = getImgLink(link)
    
    page_code = []
    for iLink in initLinks:
        s = re.findall(r"[0-9]*_[0-9]*", iLink)
        tmp = s[0].split('_')
        page_code.append((int(tmp[0]), int(tmp[1])))
    
    
    print(parseString+"\n")
    #string1 = re.findall("m=.*", parseString)
    #print(string1)
    #print("\n")
    #if string1:
    string1 = re.findall("[0-9]*_[0-9]*", parseString)
    print(string1)
    for s in string1:
        tmp = s.split('_')
        page_code.append((int(tmp[0]), int(tmp[1])))
    page_code = sorted(page_code)
    page_link = []
    page_link_tmp = []
    for code in page_code:
        page_link_tmp.append((str(code[0])+'_'+str(code[1])))
    
    index = 0
    
    if page_link_tmp:
        page_link.append(page_link_tmp[0])
    for p in page_link_tmp:
        if not page_link[index] == p:
            page_link.append(p)
            index = index + 1
    
    print(np.array(page_link_tmp))
    print(np.array(page_link))
    
    path = "./comic/"
    if not os.path.isdir(path):
        os.mkdir(path)

    path = path + series+"/"
    if not os.path.isdir(path):
        os.mkdir(path)
    
    
    
    
    print('正在下載....')
    for code in page_link:
        #print(code)
        #print(re.sub(r"[0-9]*_[0-9]*", code, initLink))
        imgUrl = re.sub(r"[0-9]*_[0-9]*", code, initLinks[0])
        imgRes = requests.get(imgUrl, headers=headers)
        
        with open(path+code.split('_')[0]+'.jpg', 'wb') as f:
            f.write(imgRes.content)
        time.sleep(1)
    print('完成')
    
    
    
"""import pandas
with lite.connect('./sqlite/comics.sqlite') as db:
    alldata = pandas.read_sql_query('select * from comics', con = db)

print('下載中....')
getComic(alldata['link'][80], alldata['series'][80])
print('完成')"""

In [ ]:
client_id = 'plcyM9HCHueeZohpFP7a41'
callback_url = 'http://127.0.0.1:5000/callback'
URL = 'https://notify-bot.line.me/oauth/authorize?response_type=code&client_id='+client_id+'&redirect_uri='+callback_url+'&scope=notify&state=NO_STATE'
print(URL)

In [ ]:
import requests

code = '4aXmld337OG3rLEEenUNyg'
client_secret = 'QfbBtdyleMoDatZlXmSa80XS0d9q2827ykpoba5aedg'
headers = {
    'Content-type': 'application/x-www-form-urlencoded'
}

payload = {
 'code':code,
 'client_id': client_id, 
 'client_secret': client_secret,
 'redirect_uri': callback_url,
 'grant_type' : 'authorization_code'   
}

res = requests.post('https://notify-bot.line.me/oauth/token', data = payload, headers = headers)

In [ ]:
res.text

In [ ]:
token = '7dNtNbMCQG4BE35yTc1ILhIabgk1B2ON5oopivArbRq'
headers = {
    'Authorization': 'Bearer '+token
}

load = {
 'message':'\n', 
}

files = {
    'imageFiles' : open('./comic/第71話/2.jpg', 'rb')
}

res = requests.post('https://notify-api.line.me/api/notify', data = load, files=files, headers = headers)
print(res.json())

In [ ]:
import time
def send_comic_page(fileAry):
    token = '7dNtNbMCQG4BE35yTc1ILhIabgk1B2ON5oopivArbRq'
    headers = {
        'Authorization': 'Bearer '+token
    }

    payload = {
     'message':'最新的漫畫', 
    }
    
    res = requests.post('https://notify-api.line.me/api/notify', data = payload, headers = headers)
    print(res.json())
    payload = {
     'message':'\n', 
    }
    for f in fileAry:
        files = {
            'imageFile': open(f, 'rb')
        }
        print('正在寄送'+f)
        #res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)
        time.sleep(1)
        print(res.json())
    #return res.json()

In [ ]:
import os, pandas
import sqlite3 as lite
def send_comics(s):
    exist = False
    with lite.connect('./sqlite/comics.sqlite') as db:
        allData = pandas.read_sql_query('select * from comics', con = db)
        exist = s in allData.values
    if(exist):
        path = './comic/'+s+'/'
        if not os.path.isdir(path):
            with lite.connect('./sqlite/comics.sqlite') as db:
                sel = pandas.read_sql_query('select series, link from comics where series = "'+s+'"', con = db)
            getComic(sel['link'][0], sel['series'][0])
        ary = []
        for f in os.listdir(path):
            ary.append(int(f.replace('.jpg', '')))
        ary.sort()
        fileAry = []
        print(ary)
        for c in ary:
            #print('{}/{}.jpg'.format(path,c))
            fileAry.append(path+str(c)+'.jpg')
        send_comic_page(fileAry)

In [ ]:
send_comics('第71話')

## 最終程式碼

In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
from opencc import OpenCC
import numpy as np
import sqlite3 as lite
import requests, re, time, os, shutil, pandas, time

selSeries = ''
token_index = 0
tokens = [
 'BnR8Jkwot7WmXYH3AFaxHEMKKyqSXybYU292L66En8a',
 'MjJokRIA4Bkg5ql09hKh4pM20Nvhq9B9EDmCoGgdDaN',
 'A5hxqTSfp6XkMT1Rrrf44WaCUQL9A92Fdg6eOQ8ADqZ',
 'd5sCkxII1Dju9FE0eAbWdjsILGdeneiQMq9ARanj2u8',
 'QfU7JVwqTLqx1gnpwUeSrjGn0ARNSvtsdXLUjoMGOhq',
 'XtcLhlVhRH7kR2Yo0QFfL78Dy31h8xseLEoN4hOm5UY',
 'zQCI84AEJcRzQVSin8PT6jB43YFGwHn6fDI48l79FB1',
 '48eL4RkIDOjebvFG6KTEQQYTCGn37I8YyKVbn7LyK7f',
 'K4iD5rM44z6FMmS8cNEhCd8jtC5haXvwRNLtmqMm5sc',
 'WXM6BjIpvFuSus2YaXdnRANiTqnIx4Ez4mKTrD284CZ',
 'icm4Vqf5wKBRDWoQH0MBGH9sJIABp1WmirEUGKhD7sW',
 'Now3hPicu6O9HY3d2j4gbAq5GQ4goSoriA8nZJKkt5m',
 'B8B9sxeuzjFIFqnpUDraAGFHaitKBrUf54raCgbu5Vg',
 'XlRs1pRSpDtVjF1pFvMw6apj3hxZxlZwKQ1Se1BiI1X',
 'YtN2gXo3yhAUZkovFIPhbKZIUI8VbSyCUD7LpKK2VLS',
 'fIoEjkm2GWNf6G30I8SebDUOEwv8FeFQVyOVderpN8p',
 'yVoZie372AhSqb9CebkKAqeMFML8HcnaYhsyxiC2gew',
 'mQQsIWIt88JPvzdUrKBG6R4mlbRdDphFu28Fo4UuCfN',
 '8yC2gJnvpIEa6ltRg4TiHgkefaT4KOYj9jSTA3ZqEwO',
 'nslOIiEIxOY59ucDAx69e4DSCIs6icsr5Un0HUK0Ekt',
 'rXTKa94IJx2VI7As4yo8p2XPpHSjA1XDX6ElbD77fNB',
 'uLtfA4CzmH15FT3ha4S9c8h97jqiSFClYPxQGyPvWIi',
 'r3msthbJ0Cbkbp4uCI6mPWOsegZa8oMY7Ith0PN87u7',
 'wksrTHL0EOQQLsZ6qJ1bBFcacjhKgJRZhtLFZTsojUS',
 'su5fplrRz53YJfM6hNC8rvibTm1FewWsWVKfDtcrccM'
]

tokens_size = len(tokens)
#print("size: "+str(tokens_size))

# Init
headers = {'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6'}

urlBase = 'https://www.manhuaren.com'
url = 'https://www.manhuaren.com/manhua-haizeiwang-onepiece/?from=/manhua-list/'
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text, 'lxml')
liList = soup.findAll("a",{"class":"chapteritem"})
comic_title = OpenCC('s2twp').convert(soup.findAll("p",{"class":"detail-main-info-title"})[0].text)

#-------------------

def check_comics():
    # 檢查資料庫最新的漫畫集數
    global comic_title
    global url
    global urlBase
    with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
        dbCheck = pandas.read_sql_query('select "series-tw", "series-cn" from comics', con = db)

    datas = []
    with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
        datas = pandas.read_sql_query('select "series-tw", "series-cn", link from comics', con = db)
  
    #print(datas)
    
    
    # 利用網路爬蟲檢查網路上最新的集數
    headers = {'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    new_comics = []
    comic_title = OpenCC('s2twp').convert(soup.findAll("p",{"class":"detail-main-info-title"})[0].text)
    for rec in soup.findAll('a', {'class', 'chapteritem'}):
        title_ch = rec.text
        title_tw = OpenCC('s2twp').convert(title_ch)
        if not (title_tw in dbCheck.values):
            new_comics.append([title_tw, title_ch,urlBase+rec.get('href')])
            print("新增"+title_tw)
    new_comics.reverse()
     
    if not(new_comics == []):
        comics = np.append(datas.values,np.array(new_comics),0)
        comic_df = pandas.DataFrame(comics, columns = ['series-tw', 'series-ch', 'link'])
        with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
            comic_df.to_sql('comics', con = db, if_exists='replace')
    #print(new_comics)
    
    return new_comics

def getImgLink(link):
    driver = webdriver.Chrome('C:\\Users\\axuy312\\Anaconda_BigData\\高階驗證碼\\driver\\chromedriver.exe')
    driver.get(link)
    imgLinks = []
    eles = driver.find_elements_by_class_name('lazy')
    for e in eles:
        imgLinks.append(e.get_attribute('src'))
    #print(imgLink)
    driver.close()
    return imgLinks

def getComic(link, series):
    global comic_title
    headers = {'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6'}
    res = requests.get(link, headers=headers)
    #print(res.text)
    
    print('更新中....')
    soup = BeautifulSoup(res.text, 'lxml')
    scriptList = soup.findAll("script")
    
    cnt = 0
    MaxCnt = 0
    Max = 0
    parseString = ''
    for script in scriptList:
        string = script.find(text=True, recursive=False)
        if not(string == None) and len(string) > Max:
            Max = len(string)
            MaxCnt = cnt
            parseString = string
        cnt = cnt + 1
    initLinks = getImgLink(link)
    
    page_code = []
    for iLink in initLinks:
        s = re.findall(r"[0-9]*_[0-9]*", iLink)
        tmp = s[0].split('_')
        page_code.append((int(tmp[0]), int(tmp[1])))
    
    
    #print(parseString+"\n")
    #string1 = re.findall("m=.*", parseString)
    #print(string1)
    #print("\n")
    #if string1:
    string1 = re.findall("[0-9]*_[0-9]*", parseString)
    #print(string1)
    for s in string1:
        tmp = s.split('_')
        page_code.append((int(tmp[0]), int(tmp[1])))
    page_code = sorted(page_code)
    page_link = []
    page_link_tmp = []
    for code in page_code:
        page_link_tmp.append((str(code[0])+'_'+str(code[1])))
    
    index = 0
    
    if page_link_tmp:
        page_link.append(page_link_tmp[0])
    for p in page_link_tmp:
        if not page_link[index] == p:
            page_link.append(p)
            index = index + 1
    
    #print(np.array(page_link_tmp))
    print(np.array(page_link))
    
    path = "./Comics"
    if not os.path.isdir(path):
        os.mkdir(path)
    
    path = path + "/" + comic_title
    if not os.path.isdir(path):
        os.mkdir(path)

    path = path + "/" + series + "/"
    if not os.path.isdir(path):
        os.mkdir(path)
    
    
    
    
    print('正在下載....')
    for code in page_link:
        #print(code)
        #print(re.sub(r"[0-9]*_[0-9]*", code, initLink))
        imgUrl = re.sub(r"[0-9]*_[0-9]*", code, initLinks[0])
        imgRes = requests.get(imgUrl, headers=headers)
        if imgRes.reason == 'Not Found':
            if re.findall(r'\.jpg', imgUrl):
                print("檔案格式更換 (jpg -> png)")
                imgUrl = re.sub(r"\.jpg", ".png", imgUrl)
            elif re.findall(r'\.png', imgUrl):
                print("檔案格式更換 (png -> jpg)")
                imgUrl = re.sub(r"\.png", ".jpg", imgUrl)
            else:
                print('抓不到檔案格式')
            imgRes = requests.get(imgUrl, headers=headers)
            if not imgRes.reason == 'OK':
                print("檔案爬不到( At getComic(link, series) "+comic_title+"/"+series+"/"+code+" URL: "+ imgUrl+" )")
        with open(path+code.split('_')[0]+'.jpg', 'wb') as f:
            f.write(imgRes.content)
        time.sleep(1)
    print('完成')

def send_comic_page(fileAry):
    global token_index
    global tokens
    global tokens_size
    global selSeries
    
    headers = {
        'Authorization': 'Bearer '+tokens[token_index]
    }

    payload = {
     'message':selSeries, 
    }
    
    res = requests.post('https://notify-api.line.me/api/notify', data = payload, headers = headers)
    print(res.json())
    page_cnt=1
    for f in fileAry:
        files = {
            'imageFile': open(f, 'rb')
        }
        print('正在寄送'+f)
        payload = {
         'message':'第'+str(page_cnt)+'頁', 
        }
        res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)
        page_cnt = page_cnt + 1
        time.sleep(1)
        tmpToken = token_index
        while res.json()['message'] == 'Image rate limit exceeded.':
            new_tkn = (token_index + 1) % tokens_size
            print("Token更換: "+tokens[token_index]+" -> "+ tokens[new_tkn])
            token_index = new_tkn
            if token_index == tmpToken:
                print('所有Token皆暫時無法使用!')
                return
            headers = {
                'Authorization': 'Bearer '+tokens[token_index]
            }
            res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)


def send_comics(s):
    global comic_title
    exist = False
    with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
        allData = pandas.read_sql_query('select * from comics', con = db)
        #print(allData)
        exist = s in allData.values
    if(exist):
        path = './Comics/'+comic_title+'/'+s+'/'
        if not os.path.isdir(path):
            with lite.connect('./sqlite/'+comic_title+'.sqlite') as db:
                sel = pandas.read_sql_query('select "series-tw", "series-cn","link" from comics where "series-tw" = "'+s+'" or "series-cn" = "'+s+'"', con = db)
            getComic(sel['link'][0], sel['series-tw'][0])
        ary = []
        for f in os.listdir(path):
            ary.append(int(f.replace('.jpg', '')))
        ary.sort()
        fileAry = []
        print(ary)
        for c in ary:
            #print('{}/{}.jpg'.format(path,c))
            fileAry.append(path+str(c)+'.jpg')
        send_comic_page(fileAry)
    else:
        print(s+"找不到( At send_comics() )")
    

#check_comics()
        
for i in range(853,860):
    selSeries = '第'+str(i)+'話'
    send_comics(selSeries)

print('結束')

更新中....
['1_8604' '2_7724' '3_2468' '4_1722' '5_4759' '6_3775' '7_8470' '8_4521'
 '9_7534' '10_4867' '11_3668' '12_5095' '13_5035' '14_5674' '15_4556'
 '16_4629']
正在下載....
檔案格式更換 (png -> jpg)
檔案格式更換 (png -> jpg)
檔案格式更換 (png -> jpg)
完成
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{'message': 'ok', 'status': 200}
正在寄送./Comics/海賊王/第853話/1.jpg
Token更換: BnR8Jkwot7WmXYH3AFaxHEMKKyqSXybYU292L66En8a -> MjJokRIA4Bkg5ql09hKh4pM20Nvhq9B9EDmCoGgdDaN
Token更換: MjJokRIA4Bkg5ql09hKh4pM20Nvhq9B9EDmCoGgdDaN -> A5hxqTSfp6XkMT1Rrrf44WaCUQL9A92Fdg6eOQ8ADqZ
正在寄送./Comics/海賊王/第853話/2.jpg
正在寄送./Comics/海賊王/第853話/3.jpg
正在寄送./Comics/海賊王/第853話/4.jpg
正在寄送./Comics/海賊王/第853話/5.jpg
正在寄送./Comics/海賊王/第853話/6.jpg
正在寄送./Comics/海賊王/第853話/7.jpg
正在寄送./Comics/海賊王/第853話/8.jpg
正在寄送./Comics/海賊王/第853話/9.jpg
正在寄送./Comics/海賊王/第853話/10.jpg
正在寄送./Comics/海賊王/第853話/11.jpg
正在寄送./Comics/海賊王/第853話/12.jpg
正在寄送./Comics/海賊王/第853話/13.jpg
正在寄送./Comics/海賊王/第853話/14.jpg
正在寄送./Comics/海賊王/第853話/15.jpg
正在寄送./Comics/海賊王/第853話/16.jpg
更新中..

In [9]:
import requests

headers = {'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh-CN;q=0.7,zh;q=0.6'}

urlBase = 'https://www.manhuaren.com'
url = 'https://manhua1031-104-250-150-12.cdnmanhua.net/1/432/451925/4_8148.jpg'
res = requests.get(url, headers = headers)
#res.encoding = 'UTF-8'
print(res.reason)

Not Found


## 以下為增加權杖

In [ ]:
client_id = 'plcyM9HCHueeZohpFP7a41'
callback_url = 'http://127.0.0.1:5000/callback'
URL = 'https://notify-bot.line.me/oauth/authorize?response_type=code&client_id='+client_id+'&redirect_uri='+callback_url+'&scope=notify&state=NO_STATE'
print(URL)

In [ ]:
import requests
import numpy as np

code = ''
while True:
    code = input("Code: ")
    if code == '#':
        break
    client_secret = 'QfbBtdyleMoDatZlXmSa80XS0d9q2827ykpoba5aedg'
    headers = {
        'Content-type': 'application/x-www-form-urlencoded'
    }

    payload = {
     'code':code,
     'client_id': client_id, 
     'client_secret': client_secret,
     'redirect_uri': callback_url,
     'grant_type' : 'authorization_code'   
    }

    res = requests.post('https://notify-bot.line.me/oauth/token', data = payload, headers = headers)
    tkn = res.json()#['access_token']
    print(res.json())
    tokens.append(tkn['access_token'])
    print("Tokens: ")
    print(np.array(tokens))

In [ ]:
cmtt = 0
print(np.array(tokens))
for t in tokens:
    cmtt = cmtt + 1
    print(str(cmtt)+": "+t)